Esta cosa debería de tomar dos archivos, una sucesión de tiempos del GD y otra del CA, y checar coincidencias. Usemos Julia solamente y tratemos de paralelizar.

In [1]:
# using HDF5, JLD

In [1]:
ListaCA=readdir("./DatosEnero2016/espigas/CA3/");
ListaGD=readdir("./DatosEnero2016/espigas/DG/");

In [2]:
EntradasYValores=Dict{Any,Any}()
Randomizados=Dict{Any,Any}()

Dict{Any,Any} with 0 entries

In [3]:
ejemploCA=readdlm("./DatosEnero2016/CA3/Ch31_46.txt");
ejemploGD=readdlm("./DatosEnero2016/DG/Ch24_27.txt");

In [7]:
function CoincidenGDaCA(GD::Array, CA::Array)
    cuantosCA=length(CA)
    cuantosGD=length(GD)
    #Unidades en frames, son 1/7022 de segundo
    desde=13
    hasta=71
    coinciden::Int64=0
    

    j=1
    
    while j<cuantosGD
       
        k=1

        while k<cuantosCA
            dift=(CA[k]-GD[j])           
            if desde<dift<hasta
                coinciden+=1
            elseif dift>hasta
                break
            end
            k+=1        
        end
        j+=1       
    end
    
    return coinciden
end

CoincidenGDaCA (generic function with 1 method)

In [8]:
CoincidenGDaCA(ejemploGD,ejemploCA)

1

In [9]:
CoincidenciasCausales=Array(Int, 1,7)
Pupu=Int32[]
laregexp=r"([0-9]{2})_([0-9]{2})"
@time for k in ListaCA, j in ListaGD
    ejemploCA=readdlm("./DatosEnero2016/espigas/CA3/$k")
    ejemploGD=readdlm("./DatosEnero2016/espigas/DG/$j")  
    
    parapa=match(laregexp, k)
    perepe=match(laregexp,j)
    #println(typeof(parapa), parapa,k,j)
    
    parteCA=[parse(Int64,s) for s=parapa.captures ]  
    parteGD=[parse(Int64,z) for z=perepe.captures ]
    
    cuantos=CoincidenGDaCA(ejemploGD,ejemploCA)
    
    Pupu=[parteGD; parteCA; [cuantos; length(ejemploGD); length(ejemploCA)]]'
    CoincidenciasCausales=vcat(CoincidenciasCausales, Pupu)
    
end

CoincidenciasCausales=CoincidenciasCausales[2:end,:];


  5.217738 seconds (7.48 M allocations: 7.813 GB, 7.48% gc time)


In [10]:
writedlm("CoincidenciasPseudoRealesPureJulia01.dat", CoincidenciasCausales)

In [11]:
sum(CoincidenciasCausales[:,5])

9021

In [36]:
extrema(CoincidenciasCausales[:,5])

(0,9)

In [31]:
function sacaintervalos(xxs::Array)
    result=zeros(xxs)
    result[1]=xxs[1]
    for j=2:length(xxs)
        result[j]=xxs[j]-xxs[j-1]
    end
    return result
end

function tiempodeintervalos(xxs::Array)
    largo=length(xxs)
    result=zeros(largo)
    result[1]=xxs[1]
    for j=1:largo-1
        result[j+1]=result[j]+xxs[j+1]
    end
    return result
end
    

tiempodeintervalos (generic function with 1 method)

In [51]:
RandomCoincide=zeros(Float32, 1,7)
Pupu=Float32[]
cuantasveces=10000
promcoincide=0
HistoGramasTotales=Dict{Array{Int}, Array}()
@time for k in ListaCA, j in ListaGD
    
    ejemploCA=readdlm("./DatosEnero2016/CA3/$k")
    ejemploGD=readdlm("./DatosEnero2016/DG/$j")
    
    parapa=match(laregexp,k)
    perepe=match(laregexp,j)
    
    parteCA=[parse(Int64,s) for s=parapa.captures ]
    parteGD=[parse(Int64,s) for s=perepe.captures ]
        
    intervalosGD=sacaintervalos(vec(ejemploGD))
    intervalosCA=sacaintervalos(vec(ejemploCA))
    
    HistoGramasTotales[[parteGD ; parteCA]]=zeros(Int,50)
    
    for t in 1:cuantasveces
        shuffle!(intervalosGD)
        shuffle!(intervalosCA)
        novoCA=tiempodeintervalos(intervalosCA)
        novoGD=tiempodeintervalos(intervalosGD)
        
        coincidencias=CoincidenGDaCA(novoGD, novoCA)
        if coincidencias<50
        HistoGramasTotales[[parteGD ; parteCA]][coincidencias+1]+=1
        end
        promcoincide+=coincidencias
        
    end
    
    promcoincide/=cuantasveces
    Pupu=[parteGD; parteCA; [promcoincide; length(ejemploGD); length(ejemploCA)]]'
    RandomCoincide=vcat(RandomCoincide, Pupu)

end
RandomCoincide=RandomCoincide[2:end]

writedlm("RevueltasCoincidenPureJulia01.dat", RandomCoincide)

  6.940165 seconds (7.88 M allocations: 7.977 GB, 9.63% gc time)


In [52]:
HistoGramasTotales

Dict{Array{Int64,N},Array{T,N}} with 15652 entries:
  [50,15,34,55] => [1,2,0,0,0,0,0,0,0,0  …  0,0,0,0,0,0,0,0,0,0]
  [30,13,45,29] => [2,1,0,0,0,0,0,0,0,0  …  0,0,0,0,0,0,0,0,0,0]
  [48,7,34,46]  => [3,0,0,0,0,0,0,0,0,0  …  0,0,0,0,0,0,0,0,0,0]
  [26,32,41,41] => [1,2,0,0,0,0,0,0,0,0  …  0,0,0,0,0,0,0,0,0,0]
  [50,16,34,48] => [2,1,0,0,0,0,0,0,0,0  …  0,0,0,0,0,0,0,0,0,0]
  [26,27,31,54] => [2,1,0,0,0,0,0,0,0,0  …  0,0,0,0,0,0,0,0,0,0]
  [51,17,34,57] => [2,1,0,0,0,0,0,0,0,0  …  0,0,0,0,0,0,0,0,0,0]
  [49,9,36,47]  => [2,1,0,0,0,0,0,0,0,0  …  0,0,0,0,0,0,0,0,0,0]
  [33,12,37,29] => [1,2,0,0,0,0,0,0,0,0  …  0,0,0,0,0,0,0,0,0,0]
  [38,11,46,35] => [1,0,1,1,0,0,0,0,0,0  …  0,0,0,0,0,0,0,0,0,0]
  [50,12,34,50] => [2,0,1,0,0,0,0,0,0,0  …  0,0,0,0,0,0,0,0,0,0]
  [50,17,51,36] => [2,1,0,0,0,0,0,0,0,0  …  0,0,0,0,0,0,0,0,0,0]
  [27,31,31,52] => [0,2,1,0,0,0,0,0,0,0  …  0,0,0,0,0,0,0,0,0,0]
  [24,24,38,54] => [2,1,0,0,0,0,0,0,0,0  …  0,0,0,0,0,0,0,0,0,0]
  [37,10,42,43] => [3,0,0,0,0,0,0,0,0,

In [44]:
HistoGramasTotales[[10,10]]=zeros(Int, 50)

50-element Array{Int64,1}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [47]:
HistoGramasTotales[[10,10]][20]=1

1

In [48]:
HistoGramasTotales

Dict{Array{Int64,N},Array{T,N}} with 1 entry:
  [10,10] => [0,0,0,0,0,0,0,0,0,0  …  0,0,0,0,0,0,0,0,0,0]